In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Feature Extraction Network  
# Conv 50@5x5: Conv -> BN -> ReLU  
# Max Pooling 2x2  
# Conv 50@5x5  
# Max Pooling 2x2  
# FCN 1350  
# FCN 160  
# 
# Patch_CT : 17x17 stride 5  
# Patch_PT : 17x17 stride 5  
# 113 subjects  

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
from Extraction import PatchExtraction
from Cluster import ClusterInitialization, ClusterMerging
import AffinityCalculation
import os
import imp
import pickle

In [ ]:
# Find GPUs
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [3]:
def patch_extraction(img):
    patches = tf.extract_image_patches(img, ksizes=[1, 17, 17, 1],
                                      strides=[1, 5, 5, 1], 
                                      rates=[1,1,1,1], padding='SAME')
    patches = tf.reshape(patches, [-1, 17*17])
    print(f"Patch Extraction Completed: {patches.shape}")
    
    return patches

* Layers for network

In [11]:
# # Layers for Network

def convnn_CT(x):
    """Convolution Neural Networks for Feature Extraction using CT Images
    
    Input
    ___
    x : (n_samples, 17, 17, 1)
    
    """
    with mirrored_strategy.scope():
        with tf.name_scope("CT"):
            x = tf.reshape(x, [-1, 17, 17, 1])

            # First Conv Layer: 17x17 to 15x15
            W_conv1 = weight_variable([3, 3, 1, 50])
            b_conv1 = bias_variable([50])
            h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)

            # First Pooling layer: 15x15 to 8x8.
            h_pool1 = max_pool_2x2(h_conv1)

            # Second Conv Layer: 8x8 to 6x6
            W_conv2 = weight_variable([3, 3, 50, 50])
            b_conv2 = bias_variable([50])
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

            # Second pooling layer: 6x6 to 3x3
            h_pool2 = max_pool_2x2(h_conv2)

            # Flatten the layer
            h_pool2_flat = tf.reshape(h_pool2, [-1, 3 * 3 * 50])

    return h_pool2_flat


def convnn_PT(x):
    """Convolution Neural Networks for Feature Extraction using PET Images
    
    Input
    ___
    x : (n_samples, 17, 17, 1)
    
    """
    with mirrored_strategy.scope():
        with tf.name_scope("PT"):
            x = tf.reshape(x, [-1, 17, 17, 1])

            # First Conv Layer: 5x5 to 3x3
            W_conv1 = weight_variable([3, 3, 1, 50])
            b_conv1 = bias_variable([50])
            h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)

            # First Pooling layer: 15x15 to 8x8.
            h_pool1 = max_pool_2x2(h_conv1)

            # Second Conv Layer: 8x8 to 6x6
            W_conv2 = weight_variable([3, 3, 50, 50])
            b_conv2 = bias_variable([50])
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

            # Second pooling layer: 6x6 to 3x3
            h_pool2 = max_pool_2x2(h_conv2)

            # Flatten the layer
            h_pool2_flat = tf.reshape(h_pool2, [-1, 3 * 3 * 50])

    return h_pool2_flat


def conv2d(x, W):
    """conv2d는 full stride를 가진 2d convolution layer를 반환(return)한다."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')


def max_pool_2x2(x):
    """max_pool_2x2는 특징들(feature map)을 2X만큼 downsample한다."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable는 주어진 shape에 대한 weight variable을 생성한다."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable 주어진 shape에 대한 bias variable을 생성한다."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fcn_dual(ct, pt):
    """Fully Connected Networks for Feature Extraction
    """
    with mirrored_strategy.scope():
        with tf.name_scope("FCN"):
        # Merge two inputs into one vector
            x = tf.concat([ct, pt], 1)

            # FCN 1
            W_fc1 = weight_variable([3 * 3 * 50 * 2, 300])
            b_fc1 = bias_variable([300])
            h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)

            # FCN 2
            W_fc2 = weight_variable([300, 150])
            b_fc2 = bias_variable([150])
            h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

    return h_fc2

In [5]:
# Training Networks
# Extract Patches
ind_CT = [[230, 380], [150, 370]]
ind_PT = [[230, 380], [150, 370]]
path = './Example'

In [14]:
dir_list = []
for root, dirs, files in os.walk(path):
    dir_list += [addr for addr in dirs if "CT" not in addr and "PT" not in addr]
print(f'Patient Numnbers: {len(dir_list)}')

feature = []

for i in range(len(dir_list)):
    
### Patch shape -> batch * col_batch * row_batch * batchxbatch
    path_single = path + '/' + dir_list[i]
    img_CT, img_PT = PatchExtraction.stackImages(path_single, ind_CT, ind_PT)
    # Initialize Features
    # Reshape the dataset
    # x_ct = tf.placeholder(tf.float32, [None, 150, 220, 1])
    # x_pt = tf.placeholder(tf.float32, [None, 150, 220, 1])
    
    img_CT = tf.constant(img_CT, dtype=tf.float32)
    img_PT = tf.constant(img_PT, dtype=tf.float32)
    
    patches_ct = patch_extraction(img_CT)
    patches_pt = patch_extraction(img_PT)

    y_ct_conv = convnn_CT(patches_ct)
    y_pt_conv = convnn_PT(patches_pt)
    
    y_fcn = fcn_dual(y_ct_conv, y_pt_conv)
    
#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
#         feature.append(sess.run(y_fcn, 
#                                 feed_dict={x_ct:img_CT, x_pt:img_PT}))

feature = y_fcn
print(feature[0].shape)
stacked = np.vstack(feature)
print(stacked.shape)

Patient Numnbers: 1
img_ct.shape: (24, 150, 220, 1)
img_pt.shape: (24, 150, 220, 1)
Patch Extraction Completed: (31680, 289)
Patch Extraction Completed: (31680, 289)
(150,)
(31680, 150)


Patch per slice : 30 * 44 = 1320  
Patch per patient : 1320 * 24 = 31680

In [22]:
# Initialize clusters
# CPU: 564.55 secs >>> GPU
# sample number : patch number
imp.reload(ClusterInitialization)
with mirrored_strategy.scope():
    c_gpu = ClusterInitialization.Clusters(stacked[:10000], 100, K_s=20, K_a=5)

Initialize Clusters...
Elapsed time of cluster_initialize : 5.180022239685059
Number of Initial Clusters: 5698
Calculate weights for all samples...
Total distance: 23131434.14
sigma_square: 115.66
Elapsed time of calculate_weights : 10.154451131820679
Calculate affinity tables for all samples...
Calculation Numbers: 16236451.0
Affinity Table Initialization Completed!
Elapsed time of aff_initialize_CPU : 477.5987136363983
KN-N Table Initialization Completed!
Cluster Initialization Completed!


In [ ]:
# Save cluster to binary file
with open('test_191025.pickle', 'wb') as f:
    pickle.dump(c_all, f)

In [ ]:
# Load cluster
with open('test.pickle', 'rb') as f:
    c_all = pickle.load(f)

Forward Pass  2
aff_cluster_loop  
Backward Pass  
select batch
calculate triplet loss
backpropagation


In [ ]:
# Loopa
while not c_all.is_finished():
    # Forward pass
    c_all.aff_cluster_loop(eta=0.1)
    # Backward pass
    